In [1]:
import json

with open('tree.json') as json_file:
    data = json.load(json_file)

In [2]:
import pandas as pd

df_INQ = pd.read_csv('df_INQ_test_cleaned.csv')
df_ingredients = pd.read_csv('df_ingredients_clean.csv')

df_INQ.drop(columns=['Unnamed: 0'], inplace= True)
df_ingredients.drop(columns=['Unnamed: 0'], inplace= True)

df_INQ_agged = df_INQ.groupby('ID', as_index=False).agg(pd.Series.tolist)
df_ingredients_agged = df_ingredients.groupby('ID', as_index=False).agg(pd.Series.tolist)

In [3]:
from itertools import permutations 

def test_include(string, set_ingr, len):
    perm = permutations(string.split(' '), len)
    
    matches = []
    
    for i in perm:
        if ' '.join(i) in set_ingr:
            matches.append(i)
    
    return matches

In [19]:
data.get('base')

{'base sauce': {},
 'beef base': {},
 'chicken base': {},
 'clam base': {},
 'crab base': {},
 'gochujang base': {},
 'gumbo base': {},
 'lobster base': {},
 'vegetable base': {}}

In [23]:
from strsimpy import Cosine

cos = Cosine(1)

1 - cos.distance('beets', 'beetas')

0.9354143466934852

In [5]:
def find(s, trees, order, steps):
    max_order = s.split(' ')
    candidates = []
    
    if order < len(s.split(' ')):
        
        for i in permutations(s.split(' '), order):
            for tree in trees:
                candidate = tree.get(' '.join(i))
                if candidate is not None:
                    steps.append((' '.join(i), order))
                    candidates.append(candidate)
 
        if not candidates:
            
            if any([not(tree.keys()) for tree in trees]):
                
                return steps[-1]
            
            max_cosine_score = float('-inf')
            
            for i in permutations(s.split(' '), order):
                for tree in trees:
                    candidates = tree.keys()
                    for key in candidates:
                        if 1 - cos.distance(' '.join(i), key) > max_cosine_score:
                            max_cosine_score = 1 - cos.distance(s, key)
                            best_key = key
                steps.append((best_key, order - 1))
            
            return steps[-1]
            
        find(s, candidates, order+1, steps)
        
    else:
        for tree in trees:
            candidate = tree.get(s)
            if candidate is not None:
                steps.append((s, order))
                
        if not steps:
            
            max_cosine_score = float('-inf')
            
            for tree in trees:
                candidates = tree.keys()
                for key in candidates:
                    if 1 - cos.distance(s, key) > max_cosine_score:
                        max_cosine_score = 1 - cos.distance(s, key)
                        best_key = key
                            
            steps.append((best_key, order - 1))
            
            return steps[-1]
            
    return steps[-1]

In [106]:
[not({}.keys()) ]

[True]

In [6]:
find('mayo', [data], 1, [])

('yam', 0)

In [86]:
df_ingredients.iloc[0]

ID                          1
Ingredients    gochujang base
Name: 0, dtype: object

In [8]:
matches = []
for i in df_ingredients.itertuples():
    matches.append(find(i.Ingredients, [data], 1, []))

In [9]:
match_col = [x[0] for x in matches]
order_col = [x[1] for x in matches]
note_col = ['Perfect' if len(x[0].split(' ')) == x[1] else 'Not perfect' for x in matches]

In [13]:
len(note_col)

26178

In [15]:
df_ingredients['match'] = match_col
df_ingredients['order'] = order_col
df_ingredients['note'] = note_col

In [22]:
df_ingredients

,ID,Ingredients,match,order,note
0,1,gochujang base,gochujang base,2,Perfect
1,1,ketchup,ketchup,1,Perfect
2,1,maple syrup,maple syrup,2,Perfect
3,1,soy sauce,soy sauce,2,Perfect
4,1,seasoned rice wine vinegar,seasoned rice wine vinegar,4,Perfect
...,...,...,...,...,...
26173,902,banana,banana,1,Perfect
26174,902,almond milk,almond milk,2,Perfect
26175,902,packet frozen acai,za'atar,0,Not perfect
26176,903,banana,banana,1,Perfect


In [24]:
df_ingredients.to_csv('ingredients_PCA_with_notes.csv')

In [19]:
not_perfect = df_ingredients[df_ingredients['note'] == 'Not perfect']

In [21]:
not_perfect.to_csv('not_perfect.csv')

In [24]:
data.get('base')

{'base sauce': {},
 'beef base': {},
 'chicken base': {},
 'clam base': {},
 'crab base': {},
 'gochujang base': {},
 'gumbo base': {},
 'lobster base': {},
 'vegetable base': {}}

In [80]:
for i in df_ingredients_agged.iloc[0, -1]:
    print(i, find(i, [data], 1, []))

gochujang base ('gochujang base', 2)
ketchup ('ketchup', 1)
maple syrup ('maple syrup', 2)
soy sauce ('soy sauce', 2)
seasoned rice wine vinegar ('seasoned rice wine vinegar', 4)
sugar ('sugar', 1)
sesame oil ('sesame oil', 2)
water ('water', 1)
ginger ('ginger', 1)
garlic ('garlic', 1)
chicken thighs ('chicken thighs', 2)
eggs ('eggs', 1)
corn flour ('corn flour', 2)
flour ('flour', 1)
soda ('soda', 1)
garlic powder ('garlic powder', 2)
onion salt ('onion salt', 2)
spring onion ('spring onion', 2)
test
white [('whitebait', 0)]
sesame seeds ('sesame seeds', 2)
mayonnaise ('mayonnaise', 1)
salt ('salt', 1)
pepper ('pepper', 1)
vegetable oil ('vegetable oil', 2)
